<a href="https://colab.research.google.com/github/Pmskabir1234/RNN/blob/main/RNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np

# Parameters
np.random.seed(42)
n_days = 365  # one year
dates = pd.date_range("2024-01-01", periods=n_days)

# Base sinusoidal seasonal pattern for temperature
t_base = 20 + 10 * np.sin(2 * np.pi * np.arange(n_days) / 365)  # seasonal pattern

# Random weather variation
daily_noise = np.random.normal(0, 1.5, n_days)
t_max = t_base + daily_noise

# t_min a bit lower but correlated
t_min = t_max - np.random.uniform(5, 10, n_days)

# Simulate rain (more rain in lower temperatures, because why not?)
rain_prob = np.clip(1 - (t_max - 10) / 20, 0, 1)
rain = np.random.binomial(1, rain_prob) * np.random.uniform(0, 20, n_days)

# Correlate tomorrow's t_max (slight autocorrelation + random noise)
tmax_tomorrow = np.roll(t_max, -1)
tmax_tomorrow[-1] = tmax_tomorrow[-2]  # last value has no tomorrow

# Add the cooling effect of rain to tomorrow’s temp (mild)
tmax_tomorrow = tmax_tomorrow - 0.3 * (rain > 5)

# Build DataFrame
df = pd.DataFrame({
    "date": dates,
    "t_max": np.round(t_max, 2),
    "t_min": np.round(t_min, 2),
    "rain": np.round(rain, 2),
    "tmax_tomorrow": np.round(tmax_tomorrow, 2)
})



In [27]:
import random

np.random.seed(0)

#initializing the weight metrices
i_weights = np.random.rand(1,2)
h_weights = np.random.rand(2,2)
o_weights = np.random.rand(2,1)

temps = df['t_max'].tail(3).to_numpy()

x0 = temps[0].reshape(1,1)
x1 = temps[1].reshape(1,1)
x2 = temps[2].reshape(1,1)

Now we will make a forward pass, will check the outputs on the function ReLu

In [28]:
#for the first input x0
xi_0 = x0 @ i_weights
xh_0 = np.maximum(0,xi_0)     # basically applying ReLu as an activation function
xo_0 = xh_0 @ o_weights

#for the second input x1
xi_1 = x1 @ i_weights
xh_1 = np.maximum(0,xh_0+xi_1)
xo_1 = xh_1 @ o_weights

#for the third input x2
xi_2 = x2 @ i_weights
xh_2 = np.maximum(0,xh_1+xi_2)
xo_2 = xh_2 @ o_weights

print(xo_0,"\n",xo_1,"\n",xo_2)


[[16.96180733]] 
 [[34.74887858]] 
 [[53.06271403]]


AS we can see in the upper code, using the ReLu activation is quite good, simce the output is increasing all the way.
Now, we will be making a full forward pass using tanh activation.

In [29]:
#weight and bias initialization
np.random.seed(0)
i_weight = np.random.rand(1,5)/5 - .1
h_weight = np.random.rand(5,5)/5 - .1
h_bias = np.random.rand(1,5)/5 - .1
o_weight = np.random.rand(5,1)*50
o_bias = np.random.rand(1,1)

outputs = np.zeros(3)
hiddens = np.zeros((3,5))
prev_hidden = None
sequence = df['t_max'].tail(3).to_numpy()

Now, we will be making a full forward pass for the last three elements

In [30]:
for i in range(3):
  x = sequence[i].reshape(1,1)
  xi = x @ i_weight
  if prev_hidden is None:
    xh = xi
  else:
    xh = xi + prev_hidden @ h_weight + h_bias
  xh = np.tanh(xh)
  prev_hidden = xh
  xo = xh @ o_weight + o_bias

  hiddens[i] = xh
  outputs[i] = xo

print(hiddens,"\n",outputs)


[[ 0.18641004  0.68127617  0.37744574  0.17171052 -0.28672832]
 [ 0.17142784  0.72727201  0.40514684  0.29150329 -0.42080554]
 [ 0.18453064  0.74572762  0.40579821  0.30570107 -0.4351531 ]] 
 [36.93716219 39.81854676 40.98894063]


/tmp/ipython-input-2499244569.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[i] = xo


In order to reduce the training errors, we have to update
our parameters. So we need to make backward pass.

In [31]:
def mse(actual,predicted):
  return np.mean((actual-predicted)**2)

def mse_grad(actual,predicted):
  return (predicted-actual)

In [32]:
actuals = df['tmax_tomorrow'].tail(3).to_numpy()

loss_grad = mse_grad(actuals,outputs)
loss_grad

array([16.67716219, 18.95854676, 20.42894063])

In [34]:
next_hidden = None
o_weight_grad, o_bias_grad, h_weight_grad, h_bias_grad, i_weight_grad = [0] * 5

for i in range(2,-1,-1):
  l_grad = loss_grad[i].reshape(1,1)
  o_weight_grad += hiddens[i][:,np.newaxis] @ l_grad
  o_bias_grad += np.mean(l_grad)

  o_grad = l_grad @ o_weight.T

  if next_hidden is None:
    h_grad = o_grad
  else:
    h_grad = o_grad + next_hidden @ h_weight.T

  tanh_deriv = 1 - hiddens[i,:][np.newaxis,:]
  h_grad = np.multiply(h_grad, tanh_deriv)

  next_hidden = h_grad

  if i>0:
    h_weight_grad  += hiddens[i-1,:][:,np.newaxis] @ h_grad
    h_bias_grad += np.mean(l_grad)

  i_weight_grad += sequence[i].reshape(1,1).T @ h_grad

In [36]:
lr = 1e-6
i_weight -= i_weight_grad * lr
h_weight -= h_weight_grad * lr
h_bias -= h_bias_grad * lr
o_weight -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

In [44]:
i_weight

array([[-0.02155589,  0.03325866, -0.00229808, -0.03103546, -0.07013609]])

In [54]:
from sklearn.preprocessing import StandardScaler
import math

PREDICTORS = ['t_max','t_min','rain']
TARGET = 'tmax_tomorrow'

scaler = StandardScaler()
split_data = (df.sample(frac=1),[int(.7*(len(df))),int(.85*(len(df)))])
(X_train,y_train),(X_val,y_val),(X_test,y_test) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

TypeError: list indices must be integers or slices, not list